# Sales & Revenue Analysis (SQL + Python)

## Objective
Analyze sales performance using SQL queries executed through Python.
This analysis focuses on revenue trends, customer behavior, and product performance
to support data-driven business decisions.

## Tools
- SQLite (SQL queries)
- Python (sqlite3, pandas)
- Jupyter Notebook

## Executive Summary
- Total revenue exceeds **$500K**, driven primarily by Home-category products.
- Revenue concentration suggests opportunities for diversification across categories.
- High-value customers are geographically distributed, reducing regional dependency risk.
- Regional performance is balanced, though targeted growth strategies could improve underperforming areas.

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

np.random.seed(42)

In [2]:
conn = sqlite3.connect('../sql/sales.db')
cursor = conn.cursor()

In [3]:
# Number of customers
n_customers = 200

# Create customers table
customers = pd.DataFrame({
    'customer_id': range(1, n_customers + 1),
    'name': [f'Customer_{i}' for i in range(1, n_customers + 1)],
    'region': np.random.choice(
        ['North', 'South', 'East', 'West'],
        size=n_customers
    ),
    'signup_date': [
        datetime.now() - timedelta(days=int(x))
        for x in np.random.randint(30, 900, size=n_customers)
    ]
})

# Write to SQLite (replace to avoid duplicates)
customers.to_sql(
    'customers',
    conn,
    if_exists='replace',
    index=False
)

customers.head()

,customer_id,name,region,signup_date
0,1,Customer_1,East,2024-05-21 15:07:32.067631
1,2,Customer_2,West,2023-08-06 15:07:32.069767
2,3,Customer_3,North,2023-12-05 15:07:32.069782
3,4,Customer_4,East,2025-04-20 15:07:32.069787
4,5,Customer_5,East,2024-06-01 15:07:32.069791


In [4]:
# Create products table
products = pd.DataFrame({
    'product_id': range(1, 21),
    'product_name': [f'Product_{i}' for i in range(1, 21)],
    'category': np.random.choice(
        ['Electronics', 'Clothing', 'Home', 'Beauty', 'Sports'],
        size=20
    ),
    'price': np.round(
        np.random.uniform(10, 500, size=20),
        2
    )
})

products.to_sql(
    'products',
    conn,
    if_exists='replace',
    index=False
)

products.head()

,product_id,product_name,category,price
0,1,Product_1,Home,21.91
1,2,Product_2,Electronics,326.28
2,3,Product_3,Electronics,96.78
3,4,Product_4,Beauty,470.82
4,5,Product_5,Home,477.43


In [5]:
# Number of orders
n_orders = 1000

orders = pd.DataFrame({
    'order_id': range(1, n_orders + 1),
    'customer_id': np.random.randint(
        1,
        n_customers + 1,
        size=n_orders
    ),
    'order_date': [
        datetime.now() - timedelta(days=int(x))
        for x in np.random.randint(1, 730, size=n_orders)
    ],
    'total_amount': np.round(
        np.random.uniform(20, 1000, size=n_orders),
        2
    )
})

orders.to_sql(
    'orders',
    conn,
    if_exists='replace',
    index=False
)

orders.head()

,order_id,customer_id,order_date,total_amount
0,1,58,2025-10-19 15:08:39.690235,453.56
1,2,67,2025-10-29 15:08:39.690707,835.23
2,3,104,2025-04-02 15:08:39.690716,676.21
3,4,174,2025-09-17 15:08:39.690723,88.53
4,5,24,2024-09-11 15:08:39.690728,647.36


In [6]:
order_items = []
order_item_id = 1

for order_id in orders['order_id']:
    n_items = np.random.randint(1, 5)
    product_ids = np.random.choice(
        products['product_id'],
        size=n_items
    )

    for product_id in product_ids:
        order_items.append({
            'order_item_id': order_item_id,
            'order_id': order_id,
            'product_id': product_id,
            'quantity': np.random.randint(1, 4)
        })
        order_item_id += 1

order_items_df = pd.DataFrame(order_items)

order_items_df.to_sql(
    'order_items',
    conn,
    if_exists='replace',
    index=False
)

order_items_df.head()

,order_item_id,order_id,product_id,quantity
0,1,1,18,2
1,2,2,17,1
2,3,2,18,2
3,4,3,12,1
4,5,4,11,1


In [7]:
conn.close()

In [8]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('../sql/sales.db')

In [9]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,customers
1,products
2,orders
3,order_items


In [10]:
query = """
SELECT 
    ROUND(SUM(total_amount), 2) AS total_revenue
FROM orders;
"""

pd.read_sql(query, conn)

,total_revenue
0,501076.4


### Total Revenue Overview

- The business generated a total revenue of **$501,076.40** over the observed period.
- This figure establishes a strong baseline for evaluating category, customer, and regional performance.
- Subsequent analyses break down this total to identify key revenue drivers.

In [11]:
query = """
SELECT
    p.category,
    ROUND(SUM(oi.quantity * p.price), 2) AS revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.category
ORDER BY revenue DESC;
"""

revenue_by_category = pd.read_sql(query, conn)
revenue_by_category

,category,revenue
0,Home,458152.64
1,Sports,303224.28
2,Beauty,299015.78
3,Electronics,277448.95
4,Clothing,143163.36


### Revenue by Product Category

- **Home products** generate the highest revenue (**$458,152.64**), significantly outperforming all other categories.
- **Sports** and **Beauty** categories contribute similar revenue levels, indicating balanced mid-tier performance.
- **Electronics**, despite typically higher unit prices, contributes less overall revenue than expected, suggesting lower sales volume.
- **Clothing** is the lowest-performing category, indicating potential issues with pricing, demand, or product mix.

**Insight:** Revenue is highly concentrated in the Home category, representing both a strength and a potential dependency risk.


In [12]:
query = """
SELECT
    c.customer_id,
    c.region,
    ROUND(SUM(o.total_amount), 2) AS total_spent
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
GROUP BY c.customer_id, c.region
ORDER BY total_spent DESC
LIMIT 10;
"""

pd.read_sql(query, conn)

,customer_id,region,total_spent
0,104,West,6718.19
1,47,West,6583.17
2,179,North,6485.68
3,117,North,6143.10
4,8,North,6102.10
5,90,South,5543.01
6,118,South,5444.30
7,1,East,5114.74
8,69,West,5062.86
9,137,North,4929.31


### Top 10 Customers by Total Spend

- The highest-spending customer generated **$6,718.19** in total revenue.
- High-value customers are distributed across multiple regions, with a slight concentration in the **West** and **North** regions.
- No single customer dominates revenue, suggesting a healthy spread among top contributors rather than over-reliance on one account.

**Insight:** Retaining and upselling to this top customer segment could significantly impact overall revenue.

In [13]:
query = """
SELECT
    c.region,
    ROUND(SUM(o.total_amount), 2) AS revenue
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
GROUP BY c.region
ORDER BY revenue DESC;
"""

pd.read_sql(query, conn)

,region,revenue
0,East,136675.78
1,West,130976.56
2,North,125245.83
3,South,108178.23


### Revenue by Region

- The **East** region generates the highest revenue (**$136,675.78**), followed closely by the **West**.
- The **South** region underperforms relative to others, generating the lowest total revenue.
- Regional differences are present but not extreme, indicating relatively balanced geographic performance.

**Insight:** Targeted regional marketing strategies could help close the performance gap, particularly in the South.